<a href="https://colab.research.google.com/github/Minakshi85/Spam-Filter-for-Quora-Questions/blob/master/P3_Spam_filter_for_Quora_questions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np

from keras.models import Sequential
#from keras.layers.recurrent import LSTM,GRU
#from keras.layers.core import Dense, Activation, Dropout
from sklearn.metrics import roc_auc_score, accuracy_score
#from sklearn.utils import shuffle
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
from keras import models
from keras.models import load_model

from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.layers import LSTM, Activation, Dense, Input, Embedding, Dropout
from keras.models import Model
from nltk import word_tokenize

from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight

In [ ]:
!wget https://www.dropbox.com/sh/kpf9z73woodfssv/AAAwZ5DDt-aHwqZFHYrZ-ZBHa/train.csv?dl=0

--2024-03-05 10:27:29--  https://www.dropbox.com/sh/kpf9z73woodfssv/AAAwZ5DDt-aHwqZFHYrZ-ZBHa/train.csv?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.6.18, 2620:100:6019:18::a27d:412
Connecting to www.dropbox.com (www.dropbox.com)|162.125.6.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /sh/raw/kpf9z73woodfssv/AAAwZ5DDt-aHwqZFHYrZ-ZBHa/train.csv [following]
--2024-03-05 10:27:29--  https://www.dropbox.com/sh/raw/kpf9z73woodfssv/AAAwZ5DDt-aHwqZFHYrZ-ZBHa/train.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc2f6c91572a6e0703741d9c2691.dl.dropboxusercontent.com/cd/0/inline/COikMw3SJvBSMflAbC2UuXhkq0x5vn5IHCZH_ukYSZo1TsFOVDMH5rtqtRX0__WEWKk8_wEnl_6XirRe2GiYAVPyXOIexNdVDouVicO-d88dRCmcz4nK4TFZ-WTGH2lmHQ0/file# [following]
--2024-03-05 10:27:30--  https://uc2f6c91572a6e0703741d9c2691.dl.dropboxusercontent.com/cd/0/inline/COikMw3SJvBSMflAbC2UuXhkq0x5vn5IHCZH_ukYSZo1Ts

In [ ]:
!wget http://nlp.stanford.edu/data/glove.42B.300d.zip

--2024-03-05 10:27:36--  http://nlp.stanford.edu/data/glove.42B.300d.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.42B.300d.zip [following]
--2024-03-05 10:27:36--  https://nlp.stanford.edu/data/glove.42B.300d.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.42B.300d.zip [following]
--2024-03-05 10:27:36--  https://downloads.cs.stanford.edu/nlp/data/glove.42B.300d.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1877800501 (1.7G) [application/zip]


In [ ]:
!unzip /content/glove.42B.300d.zip

Archive:  /content/glove.42B.300d.zip
  inflating: glove.42B.300d.txt      


In [ ]:
embedding_index ={}

f = open('/content/glove.42B.300d.txt', encoding='utf-8')
for line in f:
  values = line.split()
  word = values[0]
  coefs = np.asarray(values[1:], dtype ='float32')
  embedding_index[word] = coefs
f.close()

In [ ]:
question = pd.read_csv(r'/content/train.csv?dl=0')

In [ ]:
print(question.shape)
question.head()

(1306122, 3)


,qid,question_text,target
0,00002165364db923c7e6,How did Quebec nationalists see their province...,0
1,000032939017120e6e44,"Do you have an adopted dog, how would you enco...",0
2,0000412ca6e4628ce2cf,Why does velocity affect time? Does velocity a...,0
3,000042bf85aa498cd78e,How did Otto von Guericke used the Magdeburg h...,0
4,0000455dfa3e01eae3af,Can I convert montra helicon D to a mountain b...,0


In [ ]:
X = question['question_text']
Y = question['target']
x_train, x_test, y_train, y_test = train_test_split(X,Y,test_size=0.2)
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((1044897,), (1044897,), (261225,), (261225,))

In [ ]:
y_test.value_counts()

0    245246
1     15979
Name: target, dtype: int64

In [ ]:
#!pip install imbalanced-learn
import imblearn
from sklearn.utils.class_weight import compute_class_weight

In [ ]:

class_weight = compute_class_weight(class_weight='balanced',classes=np.unique(y_train),y=y_train)
#class_weight = imblearn.combine.compute_class_weight('balanced', np.unique(y_train), y_train)

#print(class_weight)
class_weight_dict = dict(enumerate(class_weight))
class_weight_dict

{0: 0.5330748133288983, 1: 8.058621647051565}

In [ ]:
x_train

294192     Which countries in Southeast Asia have the wea...
1109802    Is there any good scope of chemical engineerin...
1288255    What are some small Kpop companies that are tr...
1153       What are some tips when attempting to make hom...
386804     How an American/Pakistan dual national invest ...
                                 ...                        
528517     What are the defining traits of an hostile eco...
786663     I dreamt about a person whom I know in 2015 an...
1223803    Why the fuck people upvotes answers. Because o...
23268      What are the quality standards of TCBY (The Co...
566300           Which is the dankest meme according to you?
Name: question_text, Length: 1044897, dtype: object

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
sent_lens=[len(word_tokenize(x)) for x in x_train]
print(max(sent_lens))

182


In [ ]:
print(np.percentile(sent_lens,99))
np.percentile(sent_lens,95)

45.0


31.0

In [ ]:
max_len=45

tk=Tokenizer(char_level=False,split=' ')
tk.fit_on_texts(x_train)
print(x_train.shape)

(1044897,)


In [ ]:
x_train.tail(10)

907398     Other than David Aldous, who are Aaron Brown's...
923942                   Can I get a PlayStation 4 for free?
553396     What is the reason for the conflict between Sp...
72992           What sentance do you regret saying the most?
1058909    What is the best field for a mechanical engine...
528517     What are the defining traits of an hostile eco...
786663     I dreamt about a person whom I know in 2015 an...
1223803    Why the fuck people upvotes answers. Because o...
23268      What are the quality standards of TCBY (The Co...
566300           Which is the dankest meme according to you?
Name: question_text, dtype: object

In [ ]:
seq_train=tk.texts_to_sequences(x_train)
seq_test=tk.texts_to_sequences(x_test)
print(len(seq_test), len(seq_train))
seq_train[0]

261225 1044897


[30, 193, 6, 5393, 1543, 24, 1, 11601, 23, 60, 13782, 496, 376]

In [ ]:
vocab_size=len(tk.word_index)
vocab_size

195845

In [ ]:
seq_train_matrix=sequence.pad_sequences(seq_train,maxlen=max_len)
seq_test_matrix=sequence.pad_sequences(seq_test,maxlen=max_len)
seq_train_matrix.shape, seq_test_matrix.shape

((1044897, 45), (261225, 45))

In [ ]:
# creating our own embedding matrix to bring down the size to 300
# we'll use 300 D vector representation of the words from pretrained embedding index
# that we downloaded

embedding_matrix=np.zeros((vocab_size+1,300))

for word,i in tk.word_index.items():
    embed_vector=embedding_index.get(word)
    if embed_vector is not None:
        embedding_matrix[i]=embed_vector

# if there are specific words which are not present in pretrained embedding
# their weights will remain 0. if there are too many such words
# then you should probably not use pretrained embeddings


In [ ]:
# Now while creating the model for text data , we'll keep embedding layer weights
# non-trainable as we are supplying those values from our end

inputs = Input(name='text_input',shape=[max_len])
embed=Embedding(vocab_size+1,300,input_length=max_len,mask_zero=True,
                weights=[embedding_matrix],trainable=False)(inputs)
lstm_layer1 = LSTM(512)(embed)
#lstm_layer1 = LSTM(64)(embed)
drop1 = Dropout(0.2)(lstm_layer1)
lstm_layer2 = Dense(256,activation='relu')(drop1)
#lstm_layer2 = Dense(32,activation='relu')(drop1)
drop2 = Dropout(0.2)(lstm_layer2)
lstm_layer3 = Dense(128,activation='relu')(drop2)
#lstm_layer3 = Dense(16,activation='relu')(drop2)
drop3 = Dropout(0.2)(lstm_layer3)
dense1 = Dense(25,activation='relu')(lstm_layer3)
#dense1 = Dense(1,activation='relu')(lstm_layer3)
drop4 = Dropout(0.2)(dense1)
final_layer = Dense(1,activation='sigmoid')(drop4)

model = Model(inputs=inputs,outputs=final_layer)

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_input (InputLayer)     [(None, 45)]              0         
                                                                 
 embedding (Embedding)       (None, 45, 300)           58753800  
                                                                 
 lstm (LSTM)                 (None, 512)               1665024   
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense (Dense)               (None, 256)               131328    
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 128)               32896 

In [ ]:
filepath='/content/weights-{epoch:02d}-{val_loss:.4f}.h5'
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1,
                             save_best_only=True)

In [ ]:
earlystop = EarlyStopping(monitor='val_loss', patience=1,
                          verbose=1)

In [ ]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
model.fit(seq_train_matrix, y_train, batch_size=1000, epochs=1, validation_data=(seq_test_matrix, y_test),class_weight={0: 0.5328573351855138, 1: 8.108651115146436},callbacks = [earlystop, checkpoint])

  16/1045 [..............................] - ETA: 2:56:38 - loss: 0.2359 - accuracy: 0.9051

In [ ]:
pred = model.predict([seq_test_matrix], batch_size=1000, verbose=1)

262/262 [==============================] - 94s 360ms/step


In [ ]:
pred.shape

(261225, 1)

In [ ]:
# Saving the model
from tensorflow.keras import models
from tensorflow.keras.models import model_from_json
model_json = model.to_json()
with open("model.json", "w") as json_file:
  json_file.write(model_json)
model.save_weights('model_weights.h5',overwrite = True)

In [ ]:
from tensorflow.keras.models import model_from_json

json_file = open('model.json', 'r')

loaded_model_json = json_file.read()
json_file.close()

loaded_model = model_from_json(loaded_model_json)

loaded_model.summary()

loaded_model.load_weights('model_weights.h5')


Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_input (InputLayer)     [(None, 45)]              0         
                                                                 
 embedding_1 (Embedding)     (None, 45, 300)           58911600  
                                                                 
 lstm_1 (LSTM)               (None, 64)                93440     
                                                                 
 dropout_4 (Dropout)         (None, 64)                0         
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dropout_5 (Dropout)         (None, 32)                0         
                                                                 
 dense_5 (Dense)             (None, 16)                528 

In [ ]:
#model.fit(seq_train_matrix,y_train,
             # epochs=1,
              #class_weight={0: 0.5328573351855138, 1: 8.108651115146436},
              #batch_size=32,
              #validation_data=(seq_test_matrix,y_test),
              #callbacks = [earlystop, checkpoint])

In [ ]:
#best_model = load_model('/content/model_weights.h5')

In [ ]:
#best_model.summary()

In [ ]:
p = model.predict(seq_test_matrix)
roc_auc_score(y_test,p)

8164/8164 [==============================] - 213s 26ms/step


0.5

In [ ]:
p

array([[0.07242373],
       [0.07242373],
       [0.07242373],
       ...,
       [0.07242373],
       [0.07242373],
       [0.07242373]], dtype=float32)